In [1]:
import pandas as pd
import numpy as np

import scipy.io
import imageio
from tqdm import tqdm

In [2]:
df = pd.read_csv("data/parsed_training.csv")

In [3]:
df.head()

,file_name,instance_number,part_level,is_occluded,class_name,raw_name,attr_list
0,data/ADE20K_2016_07_26/images/training/h/highw...,1,0,0,sky,sky,[]
1,data/ADE20K_2016_07_26/images/training/h/highw...,2,0,0,"road, route",road,[]
2,data/ADE20K_2016_07_26/images/training/h/highw...,3,0,0,"mountain, mount",mountain,[]
3,data/ADE20K_2016_07_26/images/training/h/highw...,4,0,0,field,field,[]
4,data/ADE20K_2016_07_26/images/training/h/highw...,5,0,0,tree,trees,[]


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 538372 entries, 0 to 538371
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   file_name        538372 non-null  object
 1   instance_number  538372 non-null  int64 
 2   part_level       538372 non-null  int64 
 3   is_occluded      538372 non-null  int64 
 4   class_name       538372 non-null  object
 5   raw_name         538372 non-null  object
 6   attr_list        538372 non-null  object
dtypes: int64(3), object(4)
memory usage: 28.8+ MB


In [5]:
df.raw_name.describe()

count     538372
unique      2943
top       window
freq       42139
Name: raw_name, dtype: object

In [6]:
class_images = df.groupby(["raw_name", "file_name"]).size().groupby(level=0).size()

In [7]:
class_images[class_images >= 100].sort_values(ascending=False)

raw_name
wall        11655
floor        9332
window       8331
sky          8260
ceiling      6598
            ...  
fruits        108
blade         104
entrance      102
sand          102
statue        101
Length: 301, dtype: int64

In [8]:
df.raw_name.nunique(), df.class_name.nunique()

(2943, 2904)

In [9]:
classes = df.groupby("class_name").size()
classes[classes > 100].shape

(369,)

In [10]:
fname = pd.DataFrame(df.file_name.str.split("/").values.tolist())

In [11]:
df["image_id"] = fname[7].combine_first(fname[6]).combine_first(fname[5]).str.extract("(\d+)")

In [12]:
def get_scene(row):
    scene = "/".join([row[5] or "", row[6] or "", row[7] or ""])
    return scene[:scene.find("ADE")].rstrip("/") or row[4]

scenes = fname.apply(get_scene, axis=1)

In [13]:
df["scene"] = scenes.values

In [14]:
df.sort_values(["image_id", "instance_number"])

,file_name,instance_number,part_level,is_occluded,class_name,raw_name,attr_list,image_id,scene
277616,data/ADE20K_2016_07_26/images/training/a/airpo...,1,0,0,wall,wall,['glass'],00000001,airport_terminal
277617,data/ADE20K_2016_07_26/images/training/a/airpo...,2,0,0,wall,wall,['glass'],00000001,airport_terminal
277618,data/ADE20K_2016_07_26/images/training/a/airpo...,3,0,0,wall,wall,['glass'],00000001,airport_terminal
277619,data/ADE20K_2016_07_26/images/training/a/airpo...,4,0,0,wall,wall,[],00000001,airport_terminal
277620,data/ADE20K_2016_07_26/images/training/a/airpo...,5,0,0,wall,wall,[],00000001,airport_terminal
...,...,...,...,...,...,...,...,...,...
296752,data/ADE20K_2016_07_26/images/training/z/zoo/A...,26,0,1,"ashcan, trash can, garbage can, wastebin, ash ...",trash can,[],00020210,zoo
296753,data/ADE20K_2016_07_26/images/training/z/zoo/A...,27,0,0,"fence, fencing",fence,[],00020210,zoo
296754,data/ADE20K_2016_07_26/images/training/z/zoo/A...,28,0,0,"fence, fencing",fence,[],00020210,zoo
296755,data/ADE20K_2016_07_26/images/training/z/zoo/A...,29,0,1,"animal, animate being, beast, brute, creature,...",animal,['zebra'],00020210,zoo


In [15]:
mat = scipy.io.loadmat('data/ADE20K_2016_07_26/index_ade20k.mat')

In [16]:
classes = []
for x in mat["index"][0][0][6][0]:
    classes.append(x[0])

In [17]:
mask = imageio.imread(
    "data/ADE20K_2016_07_26/images/training/a/airport_terminal/ADE_train_00000001_seg.png"
).astype(np.uint16)

In [18]:
uniq, counts = np.unique((mask[:, :, 0] // 10) * 256 + mask[:, :, 1], return_counts=True)

In [19]:
labeled = uniq[uniq != 0] - 1  # matlab index starts from 1, it took me 4 hours to fix this LMAO
sorted(np.array(classes)[labeled])

['ashcan, trash can, garbage can, wastebin, ash bin, ash-bin, ashbin, dustbin, trash barrel, trash bin',
 'bag, traveling bag, travelling bag, grip, suitcase',
 'ceiling',
 'column, pillar',
 'elevator, lift',
 'escalator, moving staircase, moving stairway',
 'fence, fencing',
 'first floor',
 'flag',
 'floor, flooring',
 'fountain',
 'person, individual, someone, somebody, mortal, soul',
 'plant, flora, plant life',
 'pot, flowerpot',
 'seat',
 'signboard, sign',
 'streetlight, street lamp',
 'tree',
 'wall']

In [20]:
dict(zip(np.array(classes)[labeled], counts[uniq!=0]/(mask.shape[0]*mask.shape[1])))

{'ashcan, trash can, garbage can, wastebin, ash bin, ash-bin, ashbin, dustbin, trash barrel, trash bin': 0.0008389155069986979,
 'bag, traveling bag, travelling bag, grip, suitcase': 0.0007893244425455729,
 'ceiling': 0.47514088948567706,
 'column, pillar': 0.03987693786621094,
 'elevator, lift': 0.00040976206461588543,
 'escalator, moving staircase, moving stairway': 0.0028308232625325522,
 'fence, fencing': 0.002838134765625,
 'first floor': 0.0916738510131836,
 'flag': 0.0013062159220377605,
 'floor, flooring': 0.11636098225911458,
 'fountain': 0.0221710205078125,
 'person, individual, someone, somebody, mortal, soul': 0.012876192728678385,
 'plant, flora, plant life': 0.019448280334472656,
 'pot, flowerpot': 0.0022465387980143228,
 'seat': 0.009389559427897135,
 'signboard, sign': 0.007338205973307292,
 'streetlight, street lamp': 0.0012429555257161458,
 'tree': 0.01111761728922526,
 'wall': 0.17502339680989584}

In [21]:
with_pix_ratio = []
for img in tqdm(range(1, df.image_id.astype(int).max()+1)):
    curr_img_id = str(img).zfill(8)
    subset = df[df.image_id == curr_img_id]
    for level in subset.part_level.unique():
        if level == 0:
            filename = subset.file_name.values[0].replace("_atr.txt", "_seg.png")
        else:
            filename = subset.file_name.values[0].replace("_atr.txt", f"_parts_{level}.png")
        mask = imageio.imread(filename).astype(np.uint16)
        _, instances = np.unique(mask[:, :, 2], return_inverse=True)
        uniq, counts = np.unique(instances, return_counts=True)
        newdf = pd.DataFrame()
        newdf["instance_number"] = uniq
        newdf["pixel_ratio"] = counts / (mask.shape[0]*mask.shape[1])
        newdf["part_level"] = level
        newdf["image_id"] = curr_img_id
        with_pix_ratio.append(newdf)

100%|██████████| 20210/20210 [1:02:41<00:00,  5.37it/s]


In [22]:
newdf = pd.concat(with_pix_ratio)

In [23]:
newdf.head()

,instance_number,pixel_ratio,part_level,image_id
0,0,0.007080,0,00000001
1,1,0.014314,0,00000001
2,2,0.018823,0,00000001
3,3,0.013477,0,00000001
4,4,0.047316,0,00000001


In [26]:
joined = df.merge(newdf, how="left", on=["instance_number", "part_level", "image_id"])

In [32]:
joined.to_csv("data/clean_parsed_training.csv", index=False)

In [55]:
joined.groupby("scene", as_index=False).image_id.nunique().to_csv("data/scenes.csv", index=False)

In [45]:
grouped = joined.groupby("class_name", as_index=False).agg({"pixel_ratio": ["min", "mean", "max"], "instance_number": "count", "image_id": "nunique"})

In [48]:
def images_1pct(row):
    return row[row.pixel_ratio > 0.01].image_id.nunique()

images_1pct.__name__ = "images_1pct"

images_1pct_df = joined.groupby("class_name", as_index=False).apply(images_1pct)
images_1pct_df.columns = ["class_name", "images_1pct"]

In [50]:
grouped.columns = ["_".join(c).strip("_") for c in grouped.columns]

In [53]:
grouped.merge(images_1pct_df, on=["class_name"]).to_csv("data/object_stats.csv", index=False)

In [62]:
scenes = ["living_room", "bedroom", "kitchen", "bathroom"]
indoor = joined[joined.scene.isin(scenes) & (joined.part_level <= 1)]

In [63]:
indoor_g = indoor.groupby("class_name", as_index=False).agg({"pixel_ratio": ["min", "mean", "max"], "instance_number": "count", "image_id": "nunique"})

In [64]:
indoor_g.head()

class_name pixel_ratio                      \
                                               min      mean       max   
0    air conditioner, air conditioning    0.001120  0.012942  0.082801   
1                   alarm clock, alarm    0.000809  0.004668  0.031400   
2  alarm, warning device, alarm system    0.001328  0.001328  0.001328   
3                anchor, ground tackle    0.000953  0.001257  0.001560   
4      andiron, firedog, dog, dog-iron    0.001334  0.001458  0.001581   

  instance_number image_id  
            count  nunique  
0              53       52  
1              21       21  
2               1        1  
3               2        2  
4               2        1

In [65]:
images_1pct_indoor = indoor.groupby("class_name", as_index=False).apply(images_1pct)
images_1pct_indoor.columns = ["class_name", "images_1pct"]

In [66]:
indoor_g.columns = ["_".join(c).strip("_") for c in indoor_g.columns]

In [67]:
indoor_g.merge(images_1pct_indoor, on=["class_name"]).to_csv("data/house_indoor_object_stats.csv", index=False)